## Image Classification for Pneumonia -- MY SPLIT VERSION

In [1]:
# Resources
# This one uses CNN: https://www.kaggle.com/code/homayoonkhadivi/medical-diagnosis-with-cnn-transfer-learning
# The codealong with CNN
#

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' 
import tensorflow as tf
from keras import models
from keras import layers
import pathlib
import PIL
import seaborn as sns
import time
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from sklearn.model_selection import train_test_split, cross_val_score
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers import scikit_learn
from tensorflow.keras.callbacks import EarlyStopping
import shutil
import random
import glob
import os
import sys
import itertools
import warnings



## About the Data:
The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).

Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care. 

Due to the relatively small amount of validation data (16 images), I will create my own validation data instead of using it, but I will still load it in.

## Add Personal Code

In [3]:
# SOURCE: The origin of this confusion matrix code was found on medium, '
# from https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):

    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            a = cf[0,0]
            b = cf[0,1]
            c = cf[1,0]
            d = cf[1,1]
            tn = ((a / (a+b))*100).round(2).astype(str) + '%'
            fp = ((b / (a+b))*100).round(2).astype(str) + '%'
            fn = ((c / (c+d))*100).round(2).astype(str) + '%'
            tp = ((d / (c+d))*100).round(2).astype(str) + '%'
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            rwf_score = 2*precision* (recall*2) /(precision + (recall*2))
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""

    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages =  [tn,fp,fn,tp]
        # old = group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])

    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label', weight = 'bold')
        plt.xlabel('Predicted label' + stats_text, weight = 'bold')
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title,size = 20, weight = 'bold')

In [4]:
# Define Result Saving Initial Function
dfcols = ['model_name', 'Train Accuracy', 'Test Accuracy']
model_summary = pd.DataFrame(columns=dfcols)


def save_result(model_name, Train_Accuracy, Test_Accuracy):
            global model_summary
            row = [(model_name, Train_Accuracy, Test_Accuracy)]
            res = pd.DataFrame(columns = dfcols, data = row)
            yeep = [model_summary, res]
            model_summary = pd.concat(yeep)
            model_summary = model_summary.sort_values('Test Accuracy', ascending = False)
            model_summary = model_summary.drop_duplicates()
            return model_summary.round(3)

In [5]:
folder = 're-split_data/NORMAL'
path = folder

p = os.listdir(path)
pf = pd.DataFrame(p)

norm_tot = len(pf)
print(f' There are {len(pf[0])} images in the normal folder')

 There are 1575 images in the normal folder


In [6]:
folder = 're-split_data/PNEUMONIA'
path = folder
p = os.listdir(path)
pf = pd.DataFrame(p)
pneum_tot = len(pf)
pneum_weight = len(pf) / 5863
norm_weight = 1 - pneum_weight
pf
print(f' There are {len(pf[0])} images in the pneumonia folder')
print(f' there are {1576 + len(pf[0])} total images in the dataset')

 There are 4265 images in the pneumonia folder
 there are 5841 total images in the dataset


In [7]:
print(f' The weight of pneumonia is {pneum_weight}')
print(f' The weight of normal is {norm_weight}')

 The weight of pneumonia is 0.7274432884188982
 The weight of normal is 0.27255671158110184


From here, there is some code which I used to re-split the data.

In [8]:
# define test and train split percentages
'''
norm_train = norm_tot * .7
norm_test = norm_tot * .3
pneum_train = pneum_tot * .7
pneum_test = pneum_tot * .3
'''

'\nnorm_train = norm_tot * .7\nnorm_test = norm_tot * .3\npneum_train = pneum_tot * .7\npneum_test = pneum_tot * .3\n'

In [9]:
'''
pf = os.listdir('re-split_data/NORMAL')
rand_norm_files = random.sample(pf, int(norm_train))
for file in rand_norm_files:
    shutil.copy('re-split_data/NORMAL/' + file, 're-split_data/train/normal')
'''

"\npf = os.listdir('re-split_data/NORMAL')\nrand_norm_files = random.sample(pf, int(norm_train))\nfor file in rand_norm_files:\n    shutil.copy('re-split_data/NORMAL/' + file, 're-split_data/train/normal')\n"

In [10]:
'''
p1 = os.listdir('re-split_data/train/normal')
p1 = pd.DataFrame(p1)

p2 = os.listdir('re-split_data/NORMAL')
p2 = pd.DataFrame(p2)

tester_files = pd.concat([p1[0],p2[0]]).drop_duplicates(keep=False)
'''

"\np1 = os.listdir('re-split_data/train/normal')\np1 = pd.DataFrame(p1)\n\np2 = os.listdir('re-split_data/NORMAL')\np2 = pd.DataFrame(p2)\n\ntester_files = pd.concat([p1[0],p2[0]]).drop_duplicates(keep=False)\n"

In [11]:
#for file in tester_files:
    #shutil.copy('re-split_data/NORMAL/' + file, 're-split_data/test/normal')

In [12]:
'''
# The pneumonia files
pf = os.listdir('re-split_data/PNEUMONIA')

rand_Pfiles = random.sample(pf, int(pneum_train))

for file in rand_Pfiles:
    shutil.copy('re-split_data/PNEUMONIA/' + file, 're-split_data/train/pneumonia')

p3 = os.listdir('re-split_data/train/pneumonia')
p3 = pd.DataFrame(p3)

p4 = os.listdir('re-split_data/PNEUMONIA')
p4 = pd.DataFrame(p4)

tester_p =  pd.concat([p3[0],p4[0]]).drop_duplicates(keep=False)

for file in tester_p:
    shutil.copy('re-split_data/PNEUMONIA/' + file, 're-split_data/test/pneumonia')

'''

"\n# The pneumonia files\npf = os.listdir('re-split_data/PNEUMONIA')\n\nrand_Pfiles = random.sample(pf, int(pneum_train))\n\nfor file in rand_Pfiles:\n    shutil.copy('re-split_data/PNEUMONIA/' + file, 're-split_data/train/pneumonia')\n\np3 = os.listdir('re-split_data/train/pneumonia')\np3 = pd.DataFrame(p3)\n\np4 = os.listdir('re-split_data/PNEUMONIA')\np4 = pd.DataFrame(p4)\n\ntester_p =  pd.concat([p3[0],p4[0]]).drop_duplicates(keep=False)\n\nfor file in tester_p:\n    shutil.copy('re-split_data/PNEUMONIA/' + file, 're-split_data/test/pneumonia')\n\n"

In [13]:
train_size = len(os.listdir('re-split_data/train/normal')) + len(os.listdir('re-split_data/train/pneumonia'))
test_size = len(os.listdir('re-split_data/test/normal')) + len(os.listdir('re-split_data/test/pneumonia'))
print(f' There are {train_size} images in the training set, {test_size} in the test set')

 There are 4088 images in the training set, 1752 in the test set


## Generate Test and Train Images

In [14]:

# get all the data in the directory split/test , and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory('re-split_data/train',
        target_size=(64, 64), batch_size = 4089) 

test_generator =ImageDataGenerator(rescale=1./255).flow_from_directory('re-split_data/test',
        target_size=(64, 64), batch_size = 1753) 

Found 4088 images belonging to 2 classes.
Found 1752 images belonging to 2 classes.


In [15]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

In [ ]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)

In [ ]:
train_y = np.reshape(train_labels[:,0], (4088,1))
test_y = np.reshape(test_labels[:,0], (1752,1))

## Baseline Model

In [ ]:
# Build a baseline fully connected model
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

baseline = model.fit(train_img,train_y,epochs=50, batch_size=32, 
                    validation_split= 0.1, steps_per_epoch= 25)


train_loss = baseline.history['loss']
train_acc = baseline.history['accuracy']
val_loss = baseline.history['val_loss']
val_acc = baseline.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=baseline.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=baseline.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=baseline.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=baseline.epoch, y=val_acc, ax=ax2, label='val_accuracy')

In [ ]:
# Build function that builds the model so we can evaluate in sklearn
def build_model():
    model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
    model.add(layers.Dense(7, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
keras_model = scikit_learn.KerasClassifier(build_model,
                                          epochs=50,
                                          steps_per_epoch= 10,
                                          batch_size=32,
                                          verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model, train_img, train_y, cv=3)

In [ ]:
results_train = model.evaluate(train_img, train_y)

In [ ]:
results_test = model.evaluate(test_img, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('Initial Model', results_train[1], results_test[1])

In [ ]:
# TODO: Add Confusion Matrix Code/ CF, as well as SaveResult

## Convolutional Neural Network (CNN)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))  
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_images,
                    train_y,
                    epochs=50,
                    steps_per_epoch=25,
                    batch_size=32,  
                    validation_split=0.1)

In [ ]:
results_train = model.evaluate(train_images, train_y)

In [ ]:
results_test = model.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('CNN #1', results_train[1], results_test[1])

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=history.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=history.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=history.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=history.epoch, y=val_acc, ax=ax2, label='val_accuracy')

## CNN #1 Results



In [ ]:
def build_cnn():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64 ,64,  3)))  
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

    return model

In [ ]:
keras_model2 = scikit_learn.KerasClassifier(build_cnn,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model2, train_img, train_y, cv=3)

# CNN Model 2

In [ ]:
model2 = models.Sequential()

model2.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))  
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(96, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
history2 = model2.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_split= .1)

In [ ]:
results_train = model2.evaluate(train_images, train_y)

In [ ]:
results_test = model2.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('CNN #2', results_train[1], results_test[1])

In [ ]:
train_loss = history2.history['loss']
train_acc = history2.history['accuracy']
val_loss = history2.history['val_loss']
val_acc = history2.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=history2.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=history2.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=history2.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=history2.epoch, y=val_acc, ax=ax2, label='val_accuracy')

In [ ]:
def build_cnn2():
    model2 = models.Sequential()
    model2.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))  
    model2.add(layers.MaxPooling2D((2, 2)))

    model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))
    
    model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))

    model2.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))

    model2.add(layers.Flatten())
    model2.add(layers.Dense(64, activation='relu'))
    model2.add(layers.Dense(1, activation='sigmoid'))

    model2.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

    return model2

In [ ]:
keras_model3 = scikit_learn.KerasClassifier(build_cnn2,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model3, train_img, train_y, cv=3)

# CNN Model 3

For this model, I will add batch normalization and dropout.

In [ ]:
model3 = models.Sequential()

model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(32, (4, 4), activation='relu'))
model3.add(BatchNormalization())
model3.add(layers.MaxPooling2D((2, 2)))



model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(96, (3, 3), activation='relu'))
model3.add(BatchNormalization())
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu'))
model3.add(Dropout(0.15))
model3.add(layers.Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

In [ ]:
model3.summary()

In [ ]:
history3 = model3.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_split= .1)

In [ ]:
results_train = model3.evaluate(train_images, train_y)

In [ ]:
results_test = model3.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('CNN #3', results_train[1], results_test[1])

In [ ]:
train_loss = history3.history['loss']
train_acc = history3.history['accuracy']
val_loss = history3.history['val_loss']
val_acc = history3.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=history3.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=history3.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=history3.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=history3.epoch, y=val_acc, ax=ax2, label='val_accuracy')

In [ ]:
def build_cnn3():
    model3 = models.Sequential()
    model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))



    model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Flatten())
    model3.add(layers.Dense(64, activation='relu'))
    model3.add(Dropout(0.15))
    model3.add(layers.Dense(1, activation='sigmoid'))


    model3.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['accuracy'])

    return model3

In [ ]:
keras_model3 = scikit_learn.KerasClassifier(build_cnn3,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model3, train_img, train_y, cv=3)

# CNN Model 4

For this model, I will change the optimizer to adam

In [ ]:
model4 = models.Sequential()

model4.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(32, (4, 4), activation='relu'))
model4.add(BatchNormalization())
model4.add(layers.MaxPooling2D((2, 2)))


model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Conv2D(96, (3, 3), activation='relu'))
model4.add(BatchNormalization())
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Flatten())
model4.add(layers.Dense(64, activation='relu'))
model4.add(Dropout(0.15))
model4.add(layers.Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model4.summary()

In [ ]:
history4 = model4.fit(train_images,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_split= .1)

In [ ]:
results_train = model4.evaluate(train_images, train_y)

In [ ]:
results_test = model4.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('CNN #4', results_train[1], results_test[1])

In [ ]:
train_loss = history4.history['loss']
train_acc = history4.history['accuracy']
val_loss = history4.history['val_loss']
val_acc = history4.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=history4.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=history4.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=history4.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=history4.epoch, y=val_acc, ax=ax2, label='val_accuracy')

In [ ]:
def build_cnn4():
    model4 = models.Sequential()
    model4.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model4.add(layers.MaxPooling2D((2, 2)))

    model4.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model4.add(BatchNormalization())
    model4.add(layers.MaxPooling2D((2, 2)))


    model4.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model4.add(layers.MaxPooling2D((2, 2)))

    model4.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model4.add(BatchNormalization())
    model4.add(layers.MaxPooling2D((2, 2)))

    model4.add(layers.Flatten())
    model4.add(layers.Dense(64, activation='relu'))
    model4.add(Dropout(0.15))
    model4.add(layers.Dense(1, activation='sigmoid'))


    model4.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

    return model4

In [ ]:
keras_model4 = scikit_learn.KerasClassifier(build_cnn4,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model4, train_img, train_y, cv=3)

# CNN Model 5

For this model, I will add weights and steps per epoch, with fewer epochs

In [ ]:
norm_weight

In [ ]:
pneum_weight

In [ ]:
classWeight = {0:norm_weight, 1:pneum_weight}

In [ ]:
model5 = models.Sequential()

model5.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(32, (4, 4), activation='relu'))
model5.add(BatchNormalization())
model5.add(layers.MaxPooling2D((2, 2)))


model5.add(layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Conv2D(96, (3, 3), activation='relu'))
model5.add(BatchNormalization())
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(64, activation='relu'))
model5.add(Dropout(0.15))
model5.add(layers.Dense(1, activation='sigmoid'))

model5.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model5.summary()

In [ ]:
history5 = model5.fit(train_images,
                    train_y,
                    epochs=50,
                    steps_per_epoch= 100,
                    batch_size=32,
                    validation_split= .1,
                    class_weight=classWeight)

In [ ]:
results_train = model5.evaluate(train_images, train_y)

In [ ]:
results_test = model5.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('CNN #5', results_train[1], results_test[1])

In [ ]:
train_loss = history5.history['loss']
train_acc = history5.history['accuracy']
val_loss = history5.history['val_loss']
val_acc = history5.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=history5.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=history5.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=history5.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=history5.epoch, y=val_acc, ax=ax2, label='val_accuracy')

In [ ]:
def build_cnn5():
    model3 = models.Sequential()
    model3.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))


    model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model3.add(BatchNormalization())
    model3.add(layers.MaxPooling2D((2, 2)))

    model3.add(layers.Flatten())
    model3.add(layers.Dense(64, activation='relu'))
    model3.add(Dropout(0.15))
    model3.add(layers.Dense(1, activation='sigmoid'))


    model3.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

    return model3

In [ ]:
keras_model5 = scikit_learn.KerasClassifier(build_cnn5,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2,
                                            class_weight=classWeight)

In [ ]:
# Now that it is a keras model, you can cross-validate it
cross_val_score(keras_model4, train_img, train_y, cv=3)

# CNN Model 6

For this model, I will add dropout. 

In [ ]:
model6 = models.Sequential()

model6.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(32, (4, 4), activation='relu'))
model6.add(BatchNormalization())
model6.add(Dropout(0.15))
model6.add(layers.MaxPooling2D((2, 2)))


model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
model6.add(Dropout(0.10))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Conv2D(96, (3, 3), activation='relu'))
model6.add(BatchNormalization())
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Flatten())
model6.add(layers.Dense(64, activation='relu'))
model6.add(Dropout(0.15))
model6.add(layers.Dense(1, activation='sigmoid'))

model6.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
model6.summary()

In [ ]:
history6 = model6.fit(train_images,
                    train_y,
                    epochs=50,
                    steps_per_epoch= 100,
                    batch_size=32,
                    validation_split= .1,
                    class_weight=classWeight)

In [ ]:
results_train = model6.evaluate(train_images, train_y)

In [ ]:
results_test = model6.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
save_result('CNN #6', results_train[1], results_test[1])

In [ ]:
train_loss = history6.history['loss']
train_acc = history6.history['accuracy']
val_loss = history6.history['val_loss']
val_acc = history6.history['val_accuracy']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.lineplot(x=history6.epoch, y=train_loss, ax=ax1, label='train_loss')
sns.lineplot(x=history6.epoch, y=train_acc, ax=ax2, label='train_accuracy')
sns.lineplot(x=history6.epoch, y=val_loss, ax=ax1, label='val_loss')
sns.lineplot(x=history6.epoch, y=val_acc, ax=ax2, label='val_accuracy')

In [ ]:
def build_cnn6():
    model6.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(64 ,64,  3)))  
    model6.add(layers.MaxPooling2D((2, 2)))

    model6.add(layers.Conv2D(32, (4, 4), activation='relu'))
    model6.add(BatchNormalization())
    model6.add(layers.MaxPooling2D((2, 2)))


    model6.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model6.add(layers.MaxPooling2D((2, 2)))

    model6.add(layers.Conv2D(96, (3, 3), activation='relu'))
    model6.add(BatchNormalization())
    model6.add(layers.MaxPooling2D((2, 2)))

    model6.add(layers.Flatten())
    model6.add(layers.Dense(64, activation='relu'))
    model6.add(Dropout(0.15))
    model6.add(layers.Dense(1, activation='sigmoid'))

    model6.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

    return model6

In [ ]:
keras_model6 = scikit_learn.KerasClassifier(build_cnn6,
                                            epochs=50,
                                            steps_per_epoch=25,
                                            validation_split= .1,
                                            batch_size=32,
                                            verbose=2,
                                            class_weight=classWeight)